<a href="https://colab.research.google.com/github/jg4726/public-file/blob/main/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xgboost



In [ ]:
pip install datatable

In [ ]:
pip install -U -q PyDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = dt.fread("/content/drive/MyDrive/jane_street_train.jay").to_pandas()

In [ ]:
testing = data[:20000]

In [ ]:
def reduce_memory_usage(df):   
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    df = df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns}) 
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

In [ ]:
import os
from time import time
import multiprocessing
import gc

import numpy as np
from numpy import percentile
import pandas as pd

import xgboost as xgb
import datatable as dt

In [ ]:
t0=time()

testing = reduce_memory_usage(testing)
testing=testing[data.weight!=0]
print("Load data finished in %0.3fs" % (time() - t0))

# Settings
NAN_VALUE = -9999
features = [c for c in testing.columns if 'feature' in c]
targets = [c for c in testing.columns if 'resp' in c]

t0=time()
# split data into X (features) and Y (target)
X = testing.loc[:, features]
X = X.fillna(NAN_VALUE)
print("Define data features X and target Y finished in %0.3fs" % (time() - t0))

Memory usage of dataframe is 10.5286865234375 MB
Memory usage of dataframe after reduction 5.37884521484375 MB
Reduced by 48.91247637708548 % 
Load data finished in 0.079s
Define data features X and target Y finished in 0.008s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [ ]:
models=[]

n = 0
for target in targets:
    Y = (testing.loc[:, target] > 0).astype(int)
    t0=time()
    # Train model
    model = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=11,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.7,
        random_state=2020,
        tree_method='gpu_hist',
        nthread=multiprocessing.cpu_count()
    )
    model.fit(X, Y)
    models.append(model)
    print("Training model finished in %0.3fs" % (time() - t0))

XGBoostError: ignored

In [ ]:
pip install cupy-cuda92


In [ ]:
pip install pynvrtc

In [ ]:
pip install jupyter_tensorboard

  Created wheel for jupyter-tensorboard: filename=jupyter_tensorboard-0.2.0-py2.py3-none-any.whl size=15258 sha256=083a3cf171f52130f2df64dd9fddafc392fef8329bc5f1e710000a6b2d3aecfc
  Stored in directory: /root/.cache/pip/wheels/59/14/ab/6d0bce449039ebdcbf45c1aff8c19153a68bf3f0492a32620e
Successfully built jupyter-tensorboard


新的试试

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
train= data
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)


train = train.query('weight > 0').reset_index(drop = True)

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

X_train = train.loc[:, train.columns.str.contains('feature')].values
y_train = (train.loc[:, 'action']).values

In [ ]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 5000
hidden_units = [150, 150, 150]
dropout_rates = [0.5, 0.5, 0.5, 0.5]
label_smoothing = 1e-2
learning_rate = 1e-3

clf = create_mlp(
    130, 1, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf.fit(X_train, y_train, epochs=50, batch_size=5000)


models = []

models.append(clf)

opt_th = 0.5000


f = np.median
models = models[-3:]

Epoch 1/50
315/315 [==============================] - 44s 134ms/step - loss: 0.7089 - AUC: 0.5037
Epoch 2/50
315/315 [==============================] - 42s 134ms/step - loss: 0.6936 - AUC: 0.5078
Epoch 3/50
315/315 [==============================] - 43s 137ms/step - loss: 0.6930 - AUC: 0.5128
Epoch 4/50
315/315 [==============================] - 43s 137ms/step - loss: 0.6926 - AUC: 0.5175
Epoch 5/50
315/315 [==============================] - 42s 132ms/step - loss: 0.6925 - AUC: 0.5195
Epoch 6/50
315/315 [==============================] - 42s 133ms/step - loss: 0.6924 - AUC: 0.5206
Epoch 7/50
315/315 [==============================] - 42s 133ms/step - loss: 0.6923 - AUC: 0.5220
Epoch 8/50
315/315 [==============================] - 42s 134ms/step - loss: 0.6923 - AUC: 0.5224
Epoch 9/50
315/315 [==============================] - 42s 133ms/step - loss: 0.6922 - AUC: 0.5235
Epoch 10/50
315/315 [==============================] - 42s 133ms/step - loss: 0.6921 - AUC: 0.5247
Epoch 11/50
315/315

SyntaxError: ignored